In [4]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot
import pysam

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *
from scripts.sm_utils import *

In [2]:
config = load_config()
od = ''

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

## count up 5' intersecting vs. non-intersecting reads

In [32]:
df = pd.read_csv('test.tsv', sep='\t', header=None)

df = df[[3, 7, 9]]
df.columns = ['read_name', 'intersect', 'pls_id']

# assumptions about intersection formatting 
# from bedtools are OK
assert len(df.loc[(df.intersect==-1)&(df.pls_id!='.')].index) == 0

# only reporting one per read
assert len(df.read_name.unique()) == len(df.index)

df['int_pls'] = df.pls_id!='.'
df = df[['read_name', 'int_pls']].groupby('int_pls').nunique().reset_index()

df.rename({'read_name':'n_reads'}, axis=1, inplace=True)
df['n_total_reads'] = df.n_reads.sum(axis=0)
df['perc_reads'] = (df['n_reads']/df['n_total_reads'])*100

df.to_csv(output.tsv, sep='\t')

,int_pls,n_reads,n_total_reads,perc_reads
0,False,654,1000,65.4
1,True,346,1000,34.6


,int_pls,read_name
0,False,654
1,True,346


In [ ]:
## 

In [5]:
# start from Q10 bams
# config['lr']['q10_bam']
bam_file = 'test.bam'
threads = 8

In [ ]:
reverse_strand = {0: 16, 16: 0}
with pysam.AlignmentFile(input.bam, "rb",
        threads=resources.threads) as input_bam:
    with pysam.AlignmentFile(output.bam, "wb",
                             template=input_bam,
                             threads=resources.threads) as output_bam:
        for read in input_bam:
            if read.has_tag('ts') and read.flag in reverse_strand:
                if read.get_tag('ts') == '-':
                    read.flag = reverse_strand[read.flag]
                    read.set_tag('ts', '+')
                output_bam.write(read)

In [6]:
# get single bp bed coords of each read start from bam
bam = pysam.AlignmentFile(bam_file, 'rb')
with pysam.AlignmentFile(input.bam, "rb",
                threads=threads) as input_bam:
            with pysam.AlignmentFile(output.bam, "wb",
                                     template=input_bam,
                                     threads=resources.threads) as output_bam:
                for read in input_bam:
                    if read.has_tag('ts') and read.flag in reverse_strand:
                        if read.get_tag('ts') == '-':
                            read.flag = reverse_strand[read.flag]
                            read.set_tag('ts', '+')
                        output_bam.write(read)